Step 1: Go to the correct directory

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 2: Convert the .txt data file to .csv file

In [12]:
import pandas as pd
# covert txt to cvs
read_file = pd.read_csv ("/content/drive/MyDrive/movie_rec/movies.txt")
read_file.to_csv ("/content/drive/MyDrive/movie_rec/movies.csv", index=None)

read_file = pd.read_csv ("/content/drive/MyDrive/movie_rec/ratings.txt")
read_file.to_csv ("/content/drive/MyDrive/movie_rec/ratings.csv", index=None)

read_file = pd.read_csv ("/content/drive/MyDrive/movie_rec/tags.txt")
read_file.to_csv ("/content/drive/MyDrive/movie_rec/tags.csv", index=None)

Step 3: Install pyspark

In [13]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=3d38e7b972dfd04a4a3f3c88c0edbcba14d58dea3207b1008ae9762156c22e11
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


### Import libraries

In [ ]:
#https://grouplens.org/datasets/movielens/

In [16]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

### Initiate spark session

In [17]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

# 1. Load data

In [18]:
movies = spark.read.csv("/content/drive/MyDrive/movie_rec/movies.csv",header=True)
ratings = spark.read.csv("/content/drive/MyDrive/movie_rec/ratings.csv",header=True)

In [19]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [ ]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [ ]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



## Calculate sparsity

In [ ]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


## Interpret ratings

In [ ]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [ ]:
# Group data by userId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|     50|  204|
|   2858|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



## Build Out An ALS Model

In [ ]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

## Tell Spark how to tune your ALS model

In [ ]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \


# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


## Build your cross validation pipeline

In [ ]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_c58e7416bfbb


## Best Model and Best Model Parameters

In [ ]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

In [ ]:

# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 50
  MaxIter: 10
  RegParam: 0.15


In [ ]:
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.8682188942822926


In [ ]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   597|    471|   2.0| 4.1357207|
|   436|    471|   3.0| 3.6097066|
|   218|    471|   4.0| 2.9232428|
|   387|    471|   3.0| 3.0708382|
|   217|    471|   2.0| 2.7097173|
|   287|    471|   4.5| 2.9761252|
|    32|    471|   3.0| 3.7025106|
|   260|    471|   4.5| 3.5645263|
|   104|    471|   4.5| 3.5195568|
|   111|   1088|   3.0|  3.305781|
|   177|   1088|   3.5| 3.5467803|
|    41|   1088|   1.5|  2.449918|
|   387|   1088|   1.5| 2.6048868|
|   594|   1088|   4.5|  4.339765|
|   307|   1088|   3.0| 2.6175585|
|   509|   1088|   3.0| 3.1423986|
|   104|   1088|   3.0| 3.6711533|
|   268|   1238|   5.0| 3.7693748|
|   462|   1238|   3.5|  3.369685|
|   307|   1342|   2.0|  2.299802|
+------+-------+------+----------+
only showing top 20 rows



## Make Recommendations

In [ ]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[3379, 4.834906]...|
|   463|[[3379, 5.058715]...|
|   496|[[3379, 4.5754867...|
|   148|[[33649, 4.460032...|
|   540|[[3379, 5.3912764...|
|   392|[[3379, 4.696207]...|
|   243|[[67618, 5.629416...|
|    31|[[3379, 4.981336]...|
|   516|[[4429, 4.8097], ...|
|   580|[[3379, 4.8408875...|
+------+--------------------+



In [ ]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|   471|   3379| 4.834906|
|   471|   8477| 4.575323|
|   471|   7096|4.5383263|
|   471|  33649|4.5357804|
|   471|  86781|4.5016623|
|   471|   7767|4.4983916|
|   471| 171495| 4.490844|
|   471| 100714| 4.419473|
|   471|  78836|4.4031296|
|   471|  32582|4.3969116|
+------+-------+---------+



## Do the recommendations make sense?
Lets merge movie name and genres to teh recommendation matrix for interpretability.

In [ ]:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|  67618|   100|5.1405735|Strictly Sexual (...|Comedy|Drama|Romance|
|   3379|   100| 5.006642| On the Beach (1959)|               Drama|
|  33649|   100|5.0065255|  Saving Face (2004)|Comedy|Drama|Romance|
|  42730|   100|4.9775596|   Glory Road (2006)|               Drama|
|  74282|   100| 4.915519|Anne of Green Gab...|Children|Drama|Ro...|
|  26073|   100| 4.874221|Human Condition I...|           Drama|War|
| 184245|   100| 4.874221|De platte jungle ...|         Documentary|
|  84273|   100| 4.874221|Zeitgeist: Moving...|         Documentary|
|   7071|   100| 4.874221|Woman Under the I...|               Drama|
| 117531|   100| 4.874221|    Watermark (2014)|         Documentary|
+-------+------+---------+--------------------+--------------------+



In [ ]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+

